In [189]:
from konlpy.tag import Mecab
import pandas as pd
import numpy as np
import re
import pprint

In [190]:
article_df = pd.read_csv('./Changmin/article_df.csv')

In [191]:
print(article_df.head(3))

   Unnamed: 0                                  title                  date  \
0           0     野, '日 오염수 방류 반대' 결의안 단독 처리…與, 집단퇴장   2023.06.27. 오후 2:59   
1           1  '86운동권' 함운경, 야당 오염수 공세에 "반일 부추기려는 의도"   2023.06.28. 오전 9:38   
2           2          이낙연, 첫 행보로 DJ묘역 참배…“제 정치의 원점”  2023.06.28. 오전 11:15   

                                                main  section  \
0  與 "농해수위 표결 강행은 '날치기'…日 원조 대변인은 文정부"野 "국민만 불안, ...      100   
1  1985년 서울대 삼민투 위원장으로 미국 문화원 점거 사건을 주도했던 '86그룹(8...      100   
2  입국 나흘 만에 공개 행보···역할론엔 말 아껴정치 복귀 시동···盧묘역 참배·文예...      100   

                                                 url  
0  https://n.news.naver.com/mnews/article/586/000...  
1  https://n.news.naver.com/mnews/article/055/000...  
2  https://n.news.naver.com/mnews/article/011/000...  


In [192]:
# 정규식으로 \n 제거하기
import re
with open('./모두의말뭉치/stopword.txt') as f:
    stopwords = f.readlines()
stop_words = [re.sub(r'\n', '', word) for word in stopwords]
print(stop_words)

['않다', '되어다', '되다', '하다', '어떻다', '이렇다', '이다', '어제', '매일', '아', '휴', '아이구', '아이쿠', '아이고', '어', '나', '우리', '저희', '따라', '의해', '을', '를', '에', '의', '가', '으로', '로', '에게', '뿐이다', '의거하여', '근거하여', '입각하여', '기준으로', '예하면', '예를 들면', '예를 들자면', '저', '소인', '소생', '저희', '지말고', '하지마', '하지마라', '다른', '물론', '또한', '그리고', '비길수 없다', '해서는 안된다', '뿐만 아니라', '만이 아니다', '만은 아니다', '막론하고', '관계없이', '그치지 않다', '그러나', '그런데', '하지만', '든간에', '논하지 않다', '따지지 않다', '설사', '비록', '더라도', '아니면', '만 못하다', '하는 편이 낫다', '불문하고', '향하여', '향해서', '향하다', '쪽으로', '틈타', '이용하여', '타다', '오르다', '제외하고', '이 외에', '이 밖에', '하여야', '비로소', '한다면 몰라도', '외에도', '이곳', '여기', '부터', '기점으로', '따라서', '할 생각이다', '하려고하다', '이리하여', '그리하여', '그렇게 함으로써', '하지만', '일때', '할때', '앞에서', '중에서', '보는데서', '으로써', '로써', '까지', '해야한다', '일것이다', '반드시', '할줄알다', '할수있다', '할수있어', '임에 틀림없다', '한다면', '등', '등등', '제', '겨우', '단지', '다만', '할뿐', '딩동', '댕그', '대해서', '대하여', '대하면', '훨씬', '얼마나', '얼마만큼', '얼마큼', '남짓', '여', '얼마간', '약간', '다소', '좀', '조금', '다수', '몇', '얼마', '지만', '하물며', '또한', '그러나', '그렇지만', '하지만', '이외에

In [193]:
from konlpy.tag import Mecab

In [194]:
tokenizer = Mecab()

In [195]:
# 불용어 제거
def preprocessing(data):
  text_data = []

  for sentence in data:
    temp_data = []
    #- 토큰화
    temp_data = tokenizer.morphs(sentence) 
    #- 불용어 제거
    temp_data = [word for word in temp_data if not word in stop_words] 
    text_data.append(temp_data)

  text_data = list(map(' '.join, text_data))

  return text_data

In [196]:
titles = article_df['title'].apply(preprocessing)
text = article_df['main'].apply(preprocessing)

In [197]:
# preprocessed_articles = pd.concat([titles, text],axis=1, sort=False)

In [198]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

In [199]:
# 감성사전 불러오기
senti_dict = pd.read_csv('./모두의말뭉치/KnuSentiLex-master/SentiWord_Dict.txt'
                         , sep='\t'
                         , header=None)
senti_dict.head(2)

,0,1
0,(-;,1.0
1,(;_;),-1.0


In [200]:
senti_dict.columns=['word','sScore']

In [201]:
senti_dict.head(2)

,word,sScore
0,(-;,1.0
1,(;_;),-1.0


In [202]:
senti_dict['sScore'].unique()

array([ 1., -1.,  0., -2.,  2., nan])

In [203]:
senti_dict[senti_dict['sScore']=='nan']

,word,sScore


In [204]:
senti_dict[senti_dict['sScore']==2]

,word,sScore
78,가능성이 늘어나다,2.0
79,가능성이 있다고,2.0
80,가능하다,2.0
141,가볍고 상쾌하다,2.0
142,가볍고 상쾌한,2.0
...,...,...
14730,쾌적하다,2.0
14731,함박웃음,2.0
14843,득점,2.0
14844,상위,2.0


In [205]:
NaN_sScore = senti_dict[senti_dict['sScore'].isnull()]

In [206]:
NaN_sScore=NaN_sScore['word'].replace('갈등 -1', '갈등', inplace=True)

/var/folders/mr/cpgnc34x2jd9m79d26_k7yb40000gn/T/ipykernel_69022/461461767.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NaN_sScore=NaN_sScore['word'].replace('갈등 -1', '갈등', inplace=True)


In [209]:
NaN_sScore=NaN_sScore['sScore'].fillna('-1', inplace=True)

TypeError: 'NoneType' object is not subscriptable

In [208]:
NaN_sScore

In [164]:
senti_dict[senti_dict['sScore']>=1]['sScore'].replace('positive',inplace=True)

In [165]:
senti_dict

,word,sScore
0,(-;,1.0
1,(;_;),-1.0
2,(^^),1.0
3,(^-^),1.0
4,(^^*,1.0
...,...,...
14850,갈등 -1,NaN
14851,의혹,-1.0
14852,내팽개치다,-2.0
14853,횡령,-2.0
